## `pyPLNmodels`: motivation

#### Supervised learning: cell type classification


In [ ]:
import os
import sys
from contextlib import contextmanager
from pyPLNmodels import PlnLDA, plot_confusion_matrix

@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

In [ ]:
#| echo: true
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

def get_classif_error(data, y):
    data_train, data_test, y_train, y_test = train_test_split(data, y, test_size=0.33, random_state=42)
    lda = LDA()
    lda.fit(data_train, y_train)
    y_pred = lda.predict(data_test)
    return np.mean(y_pred != y_test)

### Classif error on raw counts

In [ ]:
#| echo: true
rna = load_scrna(n_samples = 1000)
print('Classif error:',get_classif_error(rna["endog"], rna["labels"]))

---

### Classif error on inferred latent space

In [ ]:
#| echo: true
from pyPLNmodels import Pln
latent_variables = Pln(rna["endog"]).fit().latent_variables
print("Classif error: ", get_classif_error(latent_variables, rna["labels"]))

### Qualitative assessment

<img src="figures/plnpca_vs_pca_french.png" width="100%">


# `pyPLNmodels`: introduction

## How to fit a model

### Like R-formula


In [ ]:
# | echo: true
from pyPLNmodels import Pln
pln = Pln.from_formula("endog ~ labels ", data = rna)
_ = pln.fit()

### Explicitly


In [ ]:
# | echo: true
from pyPLNmodels import Pln
pln = Pln(rna["endog"], exog = rna["labels_1hot"], add_const = False)
_ = pln.fit()

---

### Attributes and method
Available when calling `print(model)`:

In [ ]:
# | echo: true
print(pln)

## Insights


In [ ]:
# | echo: true
pln.show(figsize = (10, 8))

## Insights

In [ ]:
# | echo: true
pln.plot_regression_forest()

## Genes correlation


In [ ]:
# | echo: true
from pyPLNmodels import PlnNetwork
net = PlnNetwork(rna["endog"], penalty = 2000, compute_offsets_method = "logsum").fit()
fig, ax = plt.subplots(1, 1, figsize = (12, 5))
net.viz_network(ax=ax)

## predict newly observed data

`PlnLDA` allows to predict new data:

In [ ]:
#| echo: true
from pyPLNmodels import PlnLDA, plot_confusion_matrix
ntrain = 500
endog_train, endog_test = rna["endog"][:ntrain],rna["endog"][ntrain:]
labels_train, labels_test = rna["labels"][:ntrain], rna["labels"][ntrain:]
lda = PlnLDA(endog_train, clusters = labels_train).fit()

In [ ]:
pred_test = lda.predict_clusters(endog_test)
plot_confusion_matrix(pred_test, labels_test)

---


::: {.panel-tabset}

### Training


In [ ]:
#| code-line-numbers: "4-7|8-9"
#| echo: true
from pyPLNmodels import PlnLDA, plot_confusion_matrix
ntrain = 500
endog_train, endog_test = rna["endog"][:ntrain],rna["endog"][ntrain:]
labels_train, labels_test = rna["labels"][:ntrain], rna["labels"][ntrain:]
lda = PlnLDA(endog_train, clusters = labels_train).fit()
pred_test = lda.predict_clusters(endog_test)

### Confusion matrix


In [ ]:
# | echo: true
plot_confusion_matrix(pred_test, labels_test)

:::

## Zero-inflation investigation


In [ ]:
# | echo: true
import seaborn as sns
from pyPLNmodels import ZIPln
zi = ZIPln(rna["endog"]).fit()
sns.heatmap(zi.latent_prob)
plt.show()